# Project 1 - Data Analytics

## Analysis of the S&P500 stock market index and the performance of its sectors in periods of crisis for the last 20 years (2003 - 2023)

### 1. What is the S&P500 and why it is important in the US stock market?

##### The S&P 500 Index, or Standard & Poor's 500 Index, is a market-capitalization-weighted index of 500 leading publicly traded companies in the U.S. It covers approximately 80% of available market capitalisation. In other words, S&P500 tracks the stock performance of 500 large companies listed on stock exchanges in the United States and it is one of the most commonly followed equity indices.

##### The Standard & Poor's 500 Index (S&P 500) is the most commonly used benchmark for determining the state of the overall economy. Many investors also use the S&P 500 as a benchmark for their individual portfolios. The key advantage of using the S&P 500 as a benchmark is the wide market breadth of the large-cap companies included in the index. The index can provide a broad view of the economic health of the U.S.

##### It represents the stock market's performance by reporting the risks and returns of the biggest companies. Investors usually look at the S&P 500 to assess how the overall stock market is doing. This index is considered a leading U.S. economic indicator.

###### Reference: https://www.investopedia.com/terms/s/sp500.asp ,  https://www.thebalancemoney.com/what-is-the-sandp-500-3305888 , https://www.spglobal.com/spdji/en/indices/equity/sp-500/#data , https://www.spglobal.com/spdji/en/indices/equity/sp-500/#overview

### 2. Read and clean the datasets

#### 2.1. Import Dependencies

In [2]:
# Import core dependencies

import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
import datetime as dt

# Import dependencies for API

import json
import requests 
import pprint

#### 2.2. Stocks and sector information of the S&P500

In [3]:
# Read and review stocks and sector infromation of the S&P500

sp500_info_df = pd.read_csv('Resources/S&P 500 Constituents/sp500-constituents.csv')
sp500_info_df

,Ticker,Name,Industry,Sub-Industry,Headquarters Location,Date added
0,A,Agilent Technologies,Health Care,Health Care Equipment,"Santa Clara, California",6/5/2000
1,AAL,American Airlines Group,Industrials,Airlines,"Fort Worth, Texas",3/23/2015
2,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",7/9/2015
3,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",11/30/1982
4,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012
...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",10/6/1997
499,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",8/7/2001
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",12/23/2019
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",6/22/2001


In [4]:
# Clean stocks and sectors dataframe, then create a new cleaned dataframe

renamed_sp500_info_df = sp500_info_df.rename(columns={"Ticker": "Symbol", "Name": "Description", 
                                                      "Industry": "Sector", "Sub-Industry": "Sub-sector"})
clean_sp500_info_df = pd.DataFrame(renamed_sp500_info_df[["Symbol", "Description", "Sector", "Sub-sector"]])
clean_sp500_info_df

,Symbol,Description,Sector,Sub-sector
0,A,Agilent Technologies,Health Care,Health Care Equipment
1,AAL,American Airlines Group,Industrials,Airlines
2,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail
3,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals"
4,ABBV,AbbVie,Health Care,Pharmaceuticals
...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants
499,ZBH,Zimmer Biomet,Health Care,Health Care Equipment
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments
501,ZION,Zions Bancorporation,Financials,Regional Banks


In [5]:
# Identify sectors and current number of their constituents/companies within the S&P500

clean_sp500_info_df["Sector"].value_counts()

Information Technology    76
Industrials               70
Financials                67
Health Care               64
Consumer Discretionary    56
Consumer Staples          33
Utilities                 30
Real Estate               30
Materials                 29
Communication Services    25
Energy                    23
Name: Sector, dtype: int64

#### 2.3. Historical index price data S&P500

In [6]:
# Read and review historical index price data for the S&P500

sp500_index_df = pd.read_csv('Resources/S&P 500 Historical Data Index/S&P 500 Historical Data.csv')

print(sp500_index_df.info())
sp500_index_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      241 non-null    object 
 1   Price     241 non-null    object 
 2   Open      241 non-null    object 
 3   High      241 non-null    object 
 4   Low       241 non-null    object 
 5   Vol.      0 non-null      float64
 6   Change %  241 non-null    object 
dtypes: float64(1), object(6)
memory usage: 13.3+ KB
None


,Date,Price,Open,High,Low,Vol.,Change %
0,01/01/2023,"4,070.56","3,858.38","4,094.32","3,793.67",NaN,6.02%
1,01/12/2022,"3,839.50","4,089.50","4,103.05","3,764.14",NaN,-5.90%
2,01/11/2022,"4,080.11","3,911.50","4,080.11","3,695.60",NaN,5.38%
3,01/10/2022,"3,871.98","3,624.07","3,906.05","3,490.00",NaN,7.99%
4,01/09/2022,"3,585.62","3,929.90","4,119.69","3,585.40",NaN,-9.34%
...,...,...,...,...,...,...,...
236,01/05/2003,963.59,916.92,965.38,902.83,NaN,5.09%
237,01/04/2003,916.92,848.18,924.24,847.85,NaN,8.10%
238,01/03/2003,848.18,841.15,895.90,788.90,NaN,0.84%
239,01/02/2003,841.15,855.70,864.64,806.29,NaN,-1.70%


In [7]:
# Adjust datatypes for all columns and delete unnecessary columns

sp500_index_df["Date"] = pd.to_datetime(sp500_index_df["Date"])

sp500_index_df["Price"] = sp500_index_df["Price"].replace({',': ''}, regex=True).astype(float)
sp500_index_df["Open"] = sp500_index_df["Open"].replace({',': ''}, regex=True).astype(float)
sp500_index_df["High"] = sp500_index_df["High"].replace({',': ''}, regex=True).astype(float)
sp500_index_df["Low"] = sp500_index_df["Low"].replace({',': ''}, regex=True).astype(float)

sp500_index_df["Change %"] = sp500_index_df["Change %"].str.rstrip('%').astype(float) / 100

del sp500_index_df["Vol."]

sp500_index_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      241 non-null    datetime64[ns]
 1   Price     241 non-null    float64       
 2   Open      241 non-null    float64       
 3   High      241 non-null    float64       
 4   Low       241 non-null    float64       
 5   Change %  241 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 11.4 KB


In [8]:
# Cleaned data of the S&P500 price index

clean_sp500_index_df = pd.DataFrame(sp500_index_df)
clean_sp500_index_df

,Date,Price,Open,High,Low,Change %
0,2023-01-01,4070.56,3858.38,4094.32,3793.67,0.0602
1,2022-01-12,3839.50,4089.50,4103.05,3764.14,-0.0590
2,2022-01-11,4080.11,3911.50,4080.11,3695.60,0.0538
3,2022-01-10,3871.98,3624.07,3906.05,3490.00,0.0799
4,2022-01-09,3585.62,3929.90,4119.69,3585.40,-0.0934
...,...,...,...,...,...,...
236,2003-01-05,963.59,916.92,965.38,902.83,0.0509
237,2003-01-04,916.92,848.18,924.24,847.85,0.0810
238,2003-01-03,848.18,841.15,895.90,788.90,0.0084
239,2003-01-02,841.15,855.70,864.64,806.29,-0.0170


#### 2.4. Historical index price data per sector of the S&P500

In [9]:
# Read and review historical index price data per sector for the S&P500

sectors = ['Consumer Discretionary', 'Consumer Staples', 'Energy', 'Financials', 
           'Health Care', 'Industrials', 'Information Technology', 'Materials',
           'Real Estate', 'Communication Services', 'Utilities']

sectors_dic_df = {}

list_of_csv = ["S&P 500 Consumer Discretionary Historical Data",
               "S&P 500 Consumer Staples Historical Data",
               "S&P 500 Energy Historical Data",
               "S&P 500 Financials Historical Data",
               "S&P 500 Health Care Historical Data",
               "S&P 500 Industrials Historical Data",
               "S&P 500 Information Technology Historical Data",
               "S&P 500 Materials Historical Data",
               "S&P 500 Real Estate Historical Data",
               "S&P 500 Telecom Services Historical Data",
               "S&P 500 Utilities Historical Data"]

for file_name in list_of_csv:
    sectors_dic_df[file_name] = pd.read_csv(f'Resources/S&P 500 Historical Data Sectors/{file_name}.csv')
    sectors_dic_df[file_name]["Sector"] = sectors[list_of_csv.index(file_name)]  

sectors_merged_df = pd.concat(sectors_dic_df, ignore_index=True)

In [10]:
# Adjust datatypes for all columns and delete unnecessary columns

sectors_merged_df["Date"] = pd.to_datetime(sectors_merged_df["Date"])

sectors_merged_df["Price"] = sectors_merged_df["Price"].replace({',': ''}, regex=True).astype(float)
sectors_merged_df["Open"] = sectors_merged_df["Open"].replace({',': ''}, regex=True).astype(float)
sectors_merged_df["High"] = sectors_merged_df["High"].replace({',': ''}, regex=True).astype(float)
sectors_merged_df["Low"] = sectors_merged_df["Low"].replace({',': ''}, regex=True).astype(float)

sectors_merged_df["Change %"] = sectors_merged_df["Change %"].str.rstrip('%').astype(float) / 100

del sectors_merged_df["Vol."]

sectors_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2651 entries, 0 to 2650
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      2651 non-null   datetime64[ns]
 1   Price     2651 non-null   float64       
 2   Open      2651 non-null   float64       
 3   High      2651 non-null   float64       
 4   Low       2651 non-null   float64       
 5   Change %  2651 non-null   float64       
 6   Sector    2651 non-null   object        
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 145.1+ KB


In [11]:
# Cleaned data of the S&P500 price index per sector

clean_sectors_index_df = pd.DataFrame(sectors_merged_df)
clean_sectors_index_df

,Date,Price,Open,High,Low,Change %,Sector
0,2023-01-01,1150.81,1010.71,1159.39,987.46,0.1445,Consumer Discretionary
1,2022-01-12,1005.48,1139.78,1140.89,981.17,-0.1131,Consumer Discretionary
2,2022-01-11,1133.65,1144.33,1146.40,1023.30,0.0081,Consumer Discretionary
3,2022-01-10,1124.52,1121.29,1176.83,1051.35,0.0020,Consumer Discretionary
4,2022-01-09,1122.31,1212.60,1311.72,1119.27,-0.0809,Consumer Discretionary
...,...,...,...,...,...,...,...
2646,2003-01-05,111.50,101.42,112.85,99.47,0.0984,Utilities
2647,2003-01-04,101.51,93.65,102.35,93.26,0.0845,Utilities
2648,2003-01-03,93.60,89.53,95.06,87.66,0.0460,Utilities
2649,2003-01-02,89.48,94.60,95.45,84.62,-0.0531,Utilities


#### 2.5. Today's Data - Sector breakdown based on market capitalisation of the S&P500

##### API Link: https://site.financialmodelingprep.com/developer/docs/stock-api/#Python

In [12]:
# Import API Key and set all parameters for the URL

from password import stock_quote_key

symbols_list = clean_sp500_info_df["Symbol"].unique()
join_symbols_list = ",".join(symbols_list)

url = f"https://financialmodelingprep.com/api/v3/quote/{join_symbols_list}?apikey={stock_quote_key}"
url

'https://financialmodelingprep.com/api/v3/quote/A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BALL,BAX,BBWI,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BMY,BR,BRK.B,BRO,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDAY,CDNS,CDW,CE,CEG,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COO,COP,COST,CPB,CPRT,CPT,CRL,CRM,CSCO,CSGP,CSX,CTAS,CTLT,CTRA,CTSH,CTVA,CVS,CVX,CZR,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISH,DLR,DLTR,DOV,DOW,DPZ,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,ELV,EMN,EMR,ENPH,EOG,EPAM,EQIX,EQR,EQT,ES,ESS,ETN,ETR,ETSY,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FCX,FDS,FDX,FE,FFIV,FIS,FISV,FITB,FLT,FMC,FOX,FOXA,FRC,FRT,FSLR,FTNT,FTV,GD,GE,GEHC,GEN,GILD,GIS,GL,GLW,GM,GNRC,GOOG,GOOGL,GPC,GPN,GRMN,GS,GWW,HAL,HAS,HBAN,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRL,HSIC

In [13]:
# Make calls/requests to Financial Modeling Prep API to get most updated data of each stock and store response

response = requests.get(url).json()
print(json.dumps(response, indent=4, sort_keys=True))

[
    {
        "avgVolume": 1300290,
        "change": -3.95,
        "changesPercentage": -2.5371,
        "dayHigh": 155.52,
        "dayLow": 150.88,
        "earningsAnnouncement": "2023-02-20T10:59:00.000+0000",
        "eps": 4.18,
        "exchange": "NYSE",
        "marketCap": 44925965280,
        "name": "Agilent Technologies, Inc.",
        "open": 154.5,
        "pe": 36.3,
        "previousClose": 155.69,
        "price": 151.74,
        "priceAvg200": 133.38535,
        "priceAvg50": 152.4618,
        "sharesOutstanding": 296072000,
        "symbol": "A",
        "timestamp": 1675112583,
        "volume": 1401052,
        "yearHigh": 160.26,
        "yearLow": 112.52
    },
    {
        "avgVolume": 28054041,
        "change": -0.47,
        "changesPercentage": -2.8606,
        "dayHigh": 16.6,
        "dayLow": 15.9101,
        "earningsAnnouncement": "2023-01-26T13:30:00.000+0000",
        "eps": -3.671,
        "exchange": "NASDAQ",
        "marketCap": 10372419832,

In [14]:
# Dataframe created with the json response

sp500_index_today = pd.DataFrame.from_records(response)
sp500_index_today.head()

,symbol,name,price,changesPercentage,change,dayLow,dayHigh,yearHigh,yearLow,marketCap,...,exchange,volume,avgVolume,open,previousClose,eps,pe,earningsAnnouncement,sharesOutstanding,timestamp
0,A,"Agilent Technologies, Inc.",151.74,-2.5371,-3.95,150.8800,155.52,160.26,112.52,44925965280,...,NYSE,1401052,1300290,154.500,155.69,4.180,36.30,2023-02-20T10:59:00.000+0000,296072000,1675112583
1,AAL,American Airlines Group Inc.,15.96,-2.8606,-0.47,15.9101,16.60,21.42,11.65,10372419832,...,NASDAQ,23585954,28054041,16.290,16.43,-3.671,-4.35,2023-01-26T13:30:00.000+0000,649900992,1675112404
2,AAP,"Advance Auto Parts, Inc.",149.88,1.6549,2.44,147.6800,152.00,237.39,138.52,8880944556,...,NYSE,961637,1069041,147.680,147.44,7.800,19.22,2023-02-13T10:59:00.000+0000,59253700,1675112401
3,AAPL,Apple Inc.,143.00,-2.0078,-2.93,142.8500,145.53,179.61,124.17,2264576590848,...,NASDAQ,59365514,78563896,144.955,145.93,6.110,23.40,2023-02-02T21:00:00.000+0000,15836199936,1675112404
4,ABBV,AbbVie Inc.,145.65,-0.4307,-0.63,145.2600,147.12,175.91,133.05,257579112000,...,NYSE,4573369,5431975,145.870,146.28,7.500,19.42,2023-02-09T14:00:00.000+0000,1768480000,1675112595


In [15]:
# Clean today's data dataframe, then create a new cleaned dataframe

renamed_today_data = sp500_index_today.rename(columns={"symbol": "Symbol", "name": "Description", "price": "Price", 
                                                       "changesPercentage": "Change %", "marketCap": "Market Cap"})

clean_sp500_index_today = pd.DataFrame(renamed_today_data[["Symbol", "Description", "Price", "Change %", "Market Cap"]])
clean_sp500_index_today

,Symbol,Description,Price,Change %,Market Cap
0,A,"Agilent Technologies, Inc.",151.74,-2.5371,44925965280
1,AAL,American Airlines Group Inc.,15.96,-2.8606,10372419832
2,AAP,"Advance Auto Parts, Inc.",149.88,1.6549,8880944556
3,AAPL,Apple Inc.,143.00,-2.0078,2264576590848
4,ABBV,AbbVie Inc.,145.65,-0.4307,257579112000
...,...,...,...,...,...
496,YUM,"Yum! Brands, Inc.",129.04,0.7102,36349019520
497,ZBH,"Zimmer Biomet Holdings, Inc.",125.83,-0.0397,26405677160
498,ZBRA,Zebra Technologies Corporation,309.03,-2.5449,15955157094
499,ZION,"Zions Bancorporation, National Association",52.23,-2.1360,7764720720


In [16]:
# Today's data including the sector of S&P500

sp500_index_sector_today = pd.merge(clean_sp500_index_today, clean_sp500_info_df[['Symbol', 'Sector', 'Sub-sector']], 
                                    on='Symbol', how='left')

print(sp500_index_sector_today.info())
sp500_index_sector_today

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501 entries, 0 to 500
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Symbol       501 non-null    object 
 1   Description  501 non-null    object 
 2   Price        501 non-null    float64
 3   Change %     501 non-null    float64
 4   Market Cap   501 non-null    int64  
 5   Sector       501 non-null    object 
 6   Sub-sector   501 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 31.3+ KB
None


,Symbol,Description,Price,Change %,Market Cap,Sector,Sub-sector
0,A,"Agilent Technologies, Inc.",151.74,-2.5371,44925965280,Health Care,Health Care Equipment
1,AAL,American Airlines Group Inc.,15.96,-2.8606,10372419832,Industrials,Airlines
2,AAP,"Advance Auto Parts, Inc.",149.88,1.6549,8880944556,Consumer Discretionary,Automotive Retail
3,AAPL,Apple Inc.,143.00,-2.0078,2264576590848,Information Technology,"Technology Hardware, Storage & Peripherals"
4,ABBV,AbbVie Inc.,145.65,-0.4307,257579112000,Health Care,Pharmaceuticals
...,...,...,...,...,...,...,...
496,YUM,"Yum! Brands, Inc.",129.04,0.7102,36349019520,Consumer Discretionary,Restaurants
497,ZBH,"Zimmer Biomet Holdings, Inc.",125.83,-0.0397,26405677160,Health Care,Health Care Equipment
498,ZBRA,Zebra Technologies Corporation,309.03,-2.5449,15955157094,Information Technology,Electronic Equipment & Instruments
499,ZION,"Zions Bancorporation, National Association",52.23,-2.1360,7764720720,Financials,Regional Banks


#### 2.6. Inflation rate for goods and services (CPI) in the United States

In [17]:
# Read and review historical inflation rate for goods and services (CPI) in the United States

cpi_df = pd.read_csv('Resources/Inflation Rate CPI US/Inflation CPI.csv')
cpi_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            240 non-null    object 
 1   Inflation Rate  240 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.9+ KB


In [18]:
# Adjust datatypes for all columns and create a new cleaned dataframe

cpi_df["Date"] = pd.to_datetime(cpi_df["Date"])

clean_cpi_df = pd.DataFrame(cpi_df)
clean_cpi_df

,Date,Inflation Rate
0,2022-01-12,0.0645
1,2022-01-11,0.0711
2,2022-01-10,0.0775
3,2022-01-09,0.0820
4,2022-01-08,0.0826
...,...,...
235,2003-01-05,0.0206
236,2003-01-04,0.0223
237,2003-01-03,0.0302
238,2003-01-02,0.0298
